In [1]:
import os
import re
import json
import nltk
import googleapiclient.discovery

from nltk.sentiment import SentimentIntensityAnalyzer


In [2]:
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

# YouTube Data API key
API_KEY = ""  # Add your own data key here


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
def get_video_comments(video_id):
    youtube = googleapiclient.discovery.build(
        "youtube", "v3", developerKey=API_KEY)
    request = youtube.commentThreads().list(
        part="snippet",
        videoId=video_id,
        textFormat="plainText",
        maxResults=100
    )
    response = request.execute()
    comments = []
    for item in response["items"]:
        comment = item["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
        comments.append(comment)
    return comments


In [4]:
def analyze_sentiment(comments):
    positive_count = 0
    negative_count = 0
    neutral_count = 0
    for comment in comments:
        # Clean the comment text
        cleaned_comment = re.sub(r'http\S+', '', comment)  # Remove URLs
        # Perform sentiment analysis
        sentiment_scores = sid.polarity_scores(cleaned_comment)
        if sentiment_scores['compound'] >= 0.05:
            positive_count += 1
        elif sentiment_scores['compound'] <= -0.05:
            negative_count += 1
        else:
            neutral_count += 1
    total_comments = len(comments)
    print("Sentiment Analysis Results:")
    print("Positive Comments: {}%".format(
        (positive_count / total_comments) * 100))
    print("Negative Comments: {}%".format(
        (negative_count / total_comments) * 100))
    print("Neutral Comments: {}%".format(
        (neutral_count / total_comments) * 100))


In [6]:
from urllib.parse import urlparse,parse_qs


def main():
    video_url = input("Enter the YouTube video URL: ")
    video_id = parse_qs(urlparse(video_url).query).get("v")
    if video_id:
        comments = get_video_comments(video_id[0])
        analyze_sentiment(comments)
    else:
        print("Invalid YouTube video URL.")


if __name__ == "__main__":
    main()


Sentiment Analysis Results:
Positive Comments: 33.33333333333333%
Negative Comments: 0.0%
Neutral Comments: 66.66666666666666%
